# Introduction to CUDA-Q

    Outline
    
    0. Running the code snippets as standalone scripts
       0.1 Default target
       0.2 Simulation targets (single-gpu today, multi-gpu tomorrow)
       0.3 Hardware targets
    1. What is CUDA-Q?
    2. Quantum circuits via CUDA-Q kernels
        2.1 Single qubit circuit
        2.2 Multi-qubit circuit with parameters
    3. Algorithmic primitives
        3.1 cudaq.sample()
        3.2 cudaq.spin_op()
        3.3 cudaq.observe()
    4. Parameterized circuits
        4.1 Passing a single parameter
        4.2 Passing multiple lists
    5. Variational algorithms two ways
        5.1. VQE loop with expectation and optimization
        5.2. VQE wrapper
                cudaq.vqe()
    6. Single-gpu acceleration
    7. Important links for CUDA-Q

### 0. Running the code snippets as standalone scripts

Optional runtime flag:

            python <filename> --target <targetname>

Alternatively, put the following line in your code and drop the runtime --target flag.

            cudaq.set_target('targetname')

0.1 Default target (qpp as multi-threaded CPU statevector backend)

            python <filename> --target qpp-cpu


0.2 Single-gpu acceleration

    0.2.1 Statevector backend
  
            python <filename>  --target nvidia

    0.2.2 Tensornet backend
  
            python <filename>  --target tensornet
            
    0.2.3 MPS backend
  
            python <filename>  --target tensornet-mps

0.3 QPU backend if available
    
            python <filename>  --target quantinuum

### 1. CUDA-Q

    CUDA-Q is NVIDIA’s open-source platform for hybrid quantum-classical computing.

    - Built for high-performance, scalability, and ease-of-use.

    - QPU Agnostic : Integration with several different quantum hardware providers including
        -  superconducting
        -  trapped ion
        -  neutral atom
        -  photonic
        -  many others!

    - Interoperable with the modern scientific computing ecosystem​.

    - Retargetable : seamless transition from simulation to physical QPU.

    - Enabling users to develop performant and scalable applications.

#### 2. CUDA-Q Kernel

      Accelerated-node programming models often separate the accelerator-device code
      from existing CPU host code via function-level boundaries.

      Quantum kernels are Python/C++ functions that are executed on a quantum processing
      unit.

      Every quantum circuit is a kernel, but not every quantum kernel is a circuit. For
      instance, a quantum kernel can be built up from other kernels, allowing us to
      interpret a large quantum program as a sequence of subroutines or subcircuits.

      The kernels also allow things like conditional statements that are not availabe
      in quantum circuits.


#### 2.1. Single qubit circuit

In [19]:
!pip install cuda-quantum

In [ ]:
import cudaq

# We begin by defining the `Kernel` that we will construct our
# program with.
@cudaq.kernel
def my_first_kernel():
    '''
    This is our first CUDA Quantum kernel.
    '''
    # Next, we can allocate qubits to the kernel via `qvector(num_qubits)`.
    # It will return a single qubit.
    qubit = cudaq.qubit()

    # Now we can begin adding instructions to apply to this qubit!
    # Here we'll just add every non-parameterized
    # single qubit gate that is supported by CUDA Quantum.
    h(qubit)
    x(qubit)
    y(qubit)
    z(qubit)
    t(qubit)
    s(qubit)

    # Next, we add a measurement to the kernel so that we can sample
    # the measurement results on our simulator!
    mz(qubit)

# Draw the circuit
print(cudaq.draw(my_first_kernel))

# Finally, we can execute this kernel on the state vector simulator
# by calling `cudaq.sample`. This will execute the provided kernel
# `shots_count` number of times and return the sampled distribution
# as a `cudaq.SampleResult` dictionary.
result = cudaq.sample(my_first_kernel)

# Now let's take a look at the `SampleResult` we've gotten back!
print(result)

     ╭───╮╭───╮╭───╮╭───╮╭───╮╭───╮
q0 : ┤ h ├┤ x ├┤ y ├┤ z ├┤ t ├┤ s ├
     ╰───╯╰───╯╰───╯╰───╯╰───╯╰───╯

{ 0:495 1:505 }



#### 2.2. Multi-qubit circuit with input parameters

In [ ]:
import cudaq
import numpy as np

@cudaq.kernel
def my_second_kernel(N: int):
    '''
    This kernel accepts an input arguments.
    '''
    q = cudaq.qvector(N)
    h(q[0])
    ry(np.pi/3, q[2])
    x.ctrl(q[1], q[0]) # multi-control gate
    mz(q)

print(cudaq.draw(my_second_kernel, 3))
result = cudaq.sample(my_second_kernel, 3)

print(result)

         ╭───╮    ╭───╮
q0 : ────┤ h ├────┤ x ├
         ╰───╯    ╰─┬─╯
q1 : ───────────────●──
     ╭───────────╮     
q2 : ┤ ry(1.047) ├─────
     ╰───────────╯     

{ 000:403 001:120 100:339 101:138 }



###     3. Algorithmic primitives

    Algorithmic primitives are common programming patterns that have
    been implemented in the CUDA Quantum library.

    We will discuss the following three:

    3.1 cudaq.sample()
    3.2 cudaq.spin_op()
    3.3 cudaq.observable()

#### 3.1. cudaq.sample()

      The sample() function performs multiple measurements of the
      circuit(1000 shots by default) and returns a dictionary of the
      measurement outcomes along with their respective counts.

      # multi-control gates with sample

In [ ]:
import cudaq
import numpy as np

@cudaq.kernel
def my_third_kernel(N: int, theta: float):
    q = cudaq.qvector(N)
    h(q)
    ry(theta, q[2])
    x.ctrl([q[0],q[1]], q[2]) # ccx gate
    ry(1.23, q[2])
    ry(2.23, q[3])
    x.ctrl([q[0],q[1],q[2]], q[3]) # cccx gate
    mz(q)

# default number of shots is 1000
sample_result = cudaq.sample(my_third_kernel, 4, np.pi/3.0, shots_count=2000)
print(cudaq.draw(my_third_kernel, 4, np.pi/3.0))
print(sample_result)

     ╭───╮                                   
q0 : ┤ h ├───────────────●────────────────●──
     ├───┤               │                │  
q1 : ┤ h ├───────────────●────────────────●──
     ├───┤╭───────────╮╭─┴─╮╭──────────╮  │  
q2 : ┤ h ├┤ ry(1.047) ├┤ x ├┤ ry(1.23) ├──●──
     ├───┤├──────────┬╯╰───╯╰──────────╯╭─┴─╮
q3 : ┤ h ├┤ ry(2.23) ├──────────────────┤ x ├
     ╰───╯╰──────────╯                  ╰───╯

{ 1010:43 1001:48 1110:268 1101:190 0000:10 1000:10 0011:401 0001:47 0010:51 0100:5 1011:367 0110:50 1100:27 1111:29 0101:63 0111:391 }



In [ ]:
# clear results, result should now be empty
sample_result.clear()
print(sample_result)

{ }



In [ ]:
dir (sample_result)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'clear',
 'count',
 'dump',
 'expectation',
 'expectation_z',
 'get_marginal_counts',
 'get_register_counts',
 'get_sequential_data',
 'items',
 'most_probable',
 'probability',
 'register_names',
 'values']

####  3.2. cudaq.spin_op()

     
    The spin_op represents sum of Pauli tensor products.
    
    - Easy to compose larger, more complex Pauli tensor products and their sums.

Let's take the Hamitonian H such that, H  = $Z_0 \otimes I_1 + I_0 \otimes X_1 + Y_0 \otimes I_1 + Y_0 \otimes Y_1$.

In [ ]:
# Importing the spin_op
from cudaq import spin

# the obseravle
hamiltonian = spin.z(0) + spin.x(1) + spin.y(0) + spin.y(0) * spin.y(1)

# add some more terms
for i in range(2):
  hamiltonian += -2.0*spin.z(i)*spin.z(i+1)

print(hamiltonian)
print(hamiltonian.to_matrix())
print(hamiltonian.to_sparse_matrix())

[-2+0j] IZZ
[1+0j] ZII
[1+0j] YII
[1+0j] IXI
[1+0j] YYI
[-2+0j] ZZI

(-3,0)  (0,0)  (1,0)  (0,0)  (0,1)  (0,0) (-1,0)  (0,0)
 (0,0)  (1,0)  (0,0)  (1,0)  (0,0)  (0,1)  (0,0) (-1,0)
 (1,0)  (0,0)  (5,0)  (0,0)  (1,0)  (0,0)  (0,1)  (0,0)
 (0,0)  (1,0)  (0,0)  (1,0)  (0,0)  (1,0)  (0,0)  (0,1)
(0,-1)  (0,0)  (1,0)  (0,0) (-1,0)  (0,0)  (1,0)  (0,0)
 (0,0) (0,-1)  (0,0)  (1,0)  (0,0)  (3,0)  (0,0)  (1,0)
(-1,0)  (0,0) (0,-1)  (0,0)  (1,0)  (0,0) (-1,0)  (0,0)
 (0,0) (-1,0)  (0,0) (0,-1)  (0,0)  (1,0)  (0,0) (-5,0)

([(-3+0j), (1+0j), 1j, (-1+0j), (1+0j), (1+0j), 1j, (-1+0j), (1+0j), (5+0j), (1+0j), 1j, (1+0j), (1+0j), (1+0j), 1j, -1j, (1+0j), (-1+0j), (1+0j), -1j, (1+0j), (3+0j), (1+0j), (-1+0j), -1j, (1+0j), (-1+0j), (-1+0j), -1j, (1+0j), (-5+0j)], [0, 2, 4, 6, 1, 3, 5, 7, 0, 2, 4, 6, 1, 3, 5, 7, 0, 2, 4, 6, 1, 3, 5, 7, 0, 2, 4, 6, 1, 3, 5, 7], [0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 6, 7, 7, 7, 7])


In [ ]:
dir(hamiltonian)

['__add__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__isub__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__mul__',
 '__ne__',
 '__new__',
 '__radd__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmul__',
 '__rsub__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 'distribute_terms',
 'dump',
 'for_each_pauli',
 'for_each_term',
 'from_word',
 'get_coefficient',
 'get_qubit_count',
 'get_raw_data',
 'get_term_count',
 'is_identity',
 'random',
 'serialize',
 'to_matrix',
 'to_sparse_matrix',
 'to_string']

#### 3.3. cudaq.observe()

Compute the expectation value of the observable, i.e., $\bra{\psi}H\ket{\psi}$, where $H$ is of type spin_op.

In [ ]:
import cudaq
from cudaq import spin

# the observable
hamiltonian = spin.z(0) + spin.x(1) + spin.y(0) + spin.y(0)*spin.y(1)

# First we need to construct a cuda quantum kernel
@cudaq.kernel
def my_fourth_kernel():
  c = cudaq.qvector(3)
  t = cudaq.qubit()
  x(h(c))
  x.ctrl([c[0], c[1]], t)

# observe() takes the quantum circuit and the observable as input params
observe_result = cudaq.observe(my_fourth_kernel, hamiltonian)
observe_result.expectation()

0.9999999999999996

### 4. Parameterized circuits
    A quantum circuit parameterized by one or many parameters.

In [ ]:
# Example of a circuit with a single parameter of type float.
import cudaq
from cudaq import spin

# the observable
hamiltonian = 5.907 - 2.1433 * spin.x(0) * spin.x(1) \
            - 2.1433 * spin.y(0) * spin.y(1) + 0.21829 * spin.z(0) \
            - 6.125 * spin.z(1)

# parameterized cudaq kernel, the parameter is of type float
@cudaq.kernel
def kernel(theta: float):
  q = cudaq.qvector(2)
  x(q[0])
  ry(theta, q[1])
  cx(q[1], q[0])

# observe() takes the kernel, the observable and the kernel parameters
observe_result = cudaq.observe(kernel, hamiltonian, .59)
observe_result.expectation()

-1.7487948611472124

      Passing one or many lists of floats

In [ ]:
# Example of a circuit with two parameters, each is a list of floats
import cudaq
from cudaq import spin

# the observable
hamiltonian = 5.907 - 2.1433 * spin.x(0) * spin.x(1) \
            - 2.1433 * spin.y(0) * spin.y(1) + 0.21829 * spin.z(0) \
            - 6.125 * spin.z(1)

# parameterized cudaq kernel, the parameters are two lists of floats
@cudaq.kernel
def kernel(alphas: list[float], betas: list[float]):
  q = cudaq.qvector(2)
  x(q[0])
  ry(alphas[0], q[0])
  rz(betas[0], q[1])
  cx(q[1], q[0])

# observe() takes the kernel, the observable and the kernel parameters
observe_result = cudaq.observe(kernel, hamiltonian, [.59, 0.3], [0.4, 0.5])
observe_result.expectation()

-0.399386040840775

    5. Variational Algorithms

    Leverage the `cudaq.observe(...)` function in tandem with the `cudaq.optimizer`.

    One can choose an optimization strategy provided as specific sub-types of the `cudaq.optimizer`.

    The `cudaq.optimizer` can be replaced with optimizers from other python libraries (e.g. scipy).

#### 5.1 VQE loop with expectation & optimization

In [ ]:
# Import the necessary modules
import cudaq
from cudaq import spin

# Parameterized circuit with theta as the parameter
@cudaq.kernel
def kernel(theta: list[float]):
    qreg = cudaq.qvector(2)
    x(qreg[0])
    ry(theta[0], qreg[1])

# Observable
hamiltonian = spin.z(0) + spin.x(1) + spin.y(0)

# Initialize the gradient-free optimizer COBYLA
optimizer = cudaq.optimizers.COBYLA()

# Specify the number of iterations (optional)
optimizer.max_iterations = 10

def cost_function(x):
    # cudaq.observe() produces the expected value of a specified observable wrt a given parameterized ansatz at given params.
    # This value is the cost function wrt which we are optimizing.
    observeResult = cudaq.observe(kernel, hamiltonian, x)
    print (observeResult.expectation(), x)
    return observeResult.expectation()

# Carry out the optimization
opt_value, opt_theta = optimizer.optimize(dimensions=1, function=cost_function)

-0.9999999999999997 [0.0]
0.0 [1.5707963267948966]
-1.9999999999999996 [-1.5707963267948966]
-0.9999999999999997 [-3.141592653589793]
-0.9999999999999997 [0.0]
-1.7071067811865468 [-0.7853981633974483]
-1.9238795325112867 [-1.9634954084936207]
-1.9807852804032298 [-1.3744467859455345]
-1.995184726672196 [-1.6689710972195777]
-1.998795456205172 [-1.521708941582556]


#### 5.2 VQE wrapper
Combines the expectation computation with the optimization

In [ ]:
 # Import the necessary modules
 import cudaq
 from cudaq import spin

# Parameterized circuit with theta as the parameter
@cudaq.kernel
def kernel(theta: list[float]):
    qreg = cudaq.qvector(2)
    x(qreg[0])
    ry(theta[0], qreg[1])

 # Hamiltonian operator
 hamiltonian = hamiltonian = spin.z(0)

 # Initialize the gradient-free optimizer COBYLA
 optimizer = cudaq.optimizers.COBYLA()

 # Specify the number of iterations (optional)
 optimizer.max_iterations = 5

 # Carry out the optimization
 opt_value, opt_theta = cudaq.vqe(kernel=kernel,
                         spin_operator=hamiltonian,
                         optimizer=optimizer,
                         parameter_count=1)

 print(f"\nminimized <H> = {round(opt_value,16)}")
 print(f"optimal theta = {round(opt_theta[0],16)}")


minimized <H> = -1.0
optimal theta = 0.0


### 6. Single-gpu acceleration
    We demonstrate the power of GPU acceleration via GHZ state preparation.
    (About 200x faster than its CPU counterpart!)

In [ ]:
# To run as a script, use python <fname> --target nvidia
import cudaq

qubit_count = 30

cudaq.set_target("nvidia") # activates the single-gpu backend

@cudaq.kernel
def kernel(qubit_count: int):
    # Allocate our qubits.
    qvector = cudaq.qvector(qubit_count)
    # Place the first qubit in the superposition state.
    h(qvector[0])
    # Loop through the allocated qubits and apply controlled-X,
    # or CNOT, operations between them.
    for qubit in range(qubit_count - 1):
        x.ctrl(qvector[qubit], qvector[qubit + 1])
    # Measure the qubits.
    mz(qvector)

#print("Preparing GHZ state for", qubit_count, "qubits.")
counts = cudaq.sample(kernel, qubit_count)
counts.dump()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



RuntimeError: Invalid simulator requested: custatevec_fp32

During handling of the above exception, another exception occurred:

AttributeError: 'RuntimeError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

AssertionError
RuntimeError: Invalid simulator requested: custatevec_fp32

During handling of the above exception, another exception occurred:

AttributeError: 'RuntimeError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

TypeError: object of type 'NoneType' has no len()

During handling of the above exception, another exception occurred:

AttributeError: 'TypeError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

AssertionError
RuntimeError: Invalid simulator requested: custatevec_fp32

During handling of the above exception, another exception occurred:

AttributeError: 'RuntimeErro

    Important Links for CUDA-Q

    * Documentation
        https://nvidia.github.io/cuda-quantum/latest/index.html
    * Github repo
        https://github.com/NVIDIA/cuda-quantum
    * Examples
        https://nvidia.github.io/cuda-quantum/latest/using/examples/examples.html